In [ ]:
%matplotlib inline
import os
from pathlib import Path

import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
from datetime import datetime

#### Function to query geopandas dataframe using query arguments

In [ ]:
def query_sv3t_gpd(hotspot_gpd, query_args):
    solar_day = query_args.get('solar_day', None)
    if solar_day is not None:
        hotspot_gpd['solar_day_temp'] = pd.to_datetime(hotspot_gpd['solar_day']).dt.date
        hotspot_gpd = hotspot_gpd[(hotspot_gpd['solar_day_temp'] >= solar_day[0]) & (hotspot_gpd['solar_day_temp'] <= solar_day[1])]
        hotspot_gpd = hotspot_gpd.drop(['solar_day_temp'], axis=1)
        
    query_string = ''
    for k, v in query_args.items():
        if k == 'solar_day':
            continue
        query_string += ' '.join([k, v[0], v[1], ' and '])
    print(f"subseting for {query_string[:-5]} between {solar_day[0]} and {solar_day[1]}")
    if query_string == "":
        return hotspot_gpd
    return hotspot_gpd.query(query_string[:-5])

In [ ]:
def plot(df, column_name, ax,  bounds, cmap='jet', title=''):
    poly = Polygon.from_bounds(*bounds)
    df_clipped = gpd.clip(df, poly)
    df_clipped.plot(markersize=0.1, column=column_name, ax=ax)
    ax.set_title(title)
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    bound_subset = gpd.clip(world, poly)
    bound_subset.to_crs(df.crs).plot(ax=ax, color='none', edgecolor='black')

#### Set the Spatial Subset, Temporal and fieldname subset parameters
Set bounding box to represent smaller extents for rapid analysis and shorter date ranges.\
The query parameters for DEA hotsopt, ESA and EUMETSAT's Hotspot are column names within each GeoPandas Dataframe.

In [ ]:
# Set the area of interest
westlon = 110.0
southlat = -50.0
eastlon = 160.0
northlat = -10.0
bbox = (westlon, southlat, eastlon, northlat)

# set start and end time for temporal subset 
start_date = '2019-12-15' #YYYY-MM-DD
end_date = '2019-12-19'

# query arguments used in subsetting the dea hotspot
# The dea hotspot for different sensors have different fieldname. The only common fieldnames
# is 'solar_day'.
dea_hotspot_query_parameters = {
    # 'confidence': ('>=', '0'),  # The confidence level of DEA hotspot to subset above this confidence level
    # 'power': ('>=', '0.'), # The power above 0. to include all (set threshold if hotspot needs subset using Fire Radiative Power)
    'solar_day': (pd.to_datetime(start_date), pd.to_datetime(end_date))
}

# query arguments used in subsetting ESA's and EUMETSAT's FRP
s3_frp_query_parameters = {
    'F1_Fire_pixel_radiance': ('>=', '0.'), # The FRP to subset the S3 FRP, >= 0. includes all 
    'confidence': ('>=', '0.01'),
    'solar_day': (pd.to_datetime(start_date), pd.to_datetime(end_date))
}

#### Set all the geojson composite paths needed for hotspot analysis
1. Geojson composite file from Eumetsat FRP product.
2. Geojson composite file from ESA's FRP product.
3. Geojson composite file from DEA's hotspot.

In [ ]:
# path to a file containing all .geojson from eumetsat's FRP product
eumetsat_s3vt_geojson_file = Path("/home/jovyan/s3vt/workdir_s3vt_old/s3vt_eumetsat_hotspots.geojson")
# Path to a file containing all .geojson from ESA's FRP product
esa_s3vt_geojson_file = Path("/home/jovyan/s3vt/s3vt_hotspots.geojson")
# Path to a file containing DEA hotspots from WFS query
dea_hotspot_geojson_file = Path("/home/jovyan/s3vt/DEAHotspots_hotspots.geojson")

#### Load spatial subset of FRP products

In [ ]:
eumetsat_hotspot_gdf = gpd.read_file(eumetsat_s3vt_geojson_file, bbox=bbox)
esa_hotspot_gdf = gpd.read_file(esa_s3vt_geojson_file, bbox=bbox)
dea_hotspot_gdf = gpd.read_file(dea_hotspot_geojson_file, bbox=bbox)

#### Load the samples items from FRP products.

In [ ]:
# get samples from eumetsat FRP product
eumetsat_hotspot_gdf.head()

In [ ]:
for col in eumetsat_hotspot_gdf.columns:
    print(col)

In [ ]:
# get samples from ESA's FRP product
esa_hotspot_gdf.head()

In [ ]:
for col in esa_hotspot_gdf.columns:
    print(col)

In [ ]:
# get samples from DEA's hotspot
dea_hotspot_gdf.head()

###### There is difference in fields between ESA's and EUMETSAT's FRP

In [ ]:
# get the difference of fieldnames between Eumetsat and ESA's FRP
set(eumetsat_hotspot_gdf.columns) - set(esa_hotspot_gdf.columns)

#### Set FRP's GPD field of `solar_day`, `datetime` and `date` columns to pandas datetime object

In [ ]:
# set dea solar_day and datetime to pandas datetime
dea_hotspot_gdf['datetime'] = pd.to_datetime(dea_hotspot_gdf['datetime'])
dea_hotspot_gdf['solar_day'] = pd.to_datetime(dea_hotspot_gdf['solar_day'])
dea_hotspot_gdf['power'] = dea_hotspot_gdf['power'].astype('float32')   # power was not in floating point 

In [ ]:
# set eumetsat solar_day and date to pandas datetime
eumetsat_hotspot_gdf['datetime'] = pd.to_datetime(eumetsat_hotspot_gdf['date'])
eumetsat_hotspot_gdf['solar_day'] = pd.to_datetime(eumetsat_hotspot_gdf['solar_day'])


In [ ]:
# set esa solar_day and date to pandas datetime
esa_hotspot_gdf['datetime'] = pd.to_datetime(esa_hotspot_gdf['date'])
esa_hotspot_gdf['solar_day'] = pd.to_datetime(esa_hotspot_gdf['solar_day'])

#### Create `satellite_sensor_product` field which is unique for sensor and platforms and product origin (DEA, ESA, EUMETSAT)

In [ ]:
esa_hotspot_gdf['satellite_sensor_product'] = esa_hotspot_gdf['satellite']+'_'+esa_hotspot_gdf['sensor']+'_ESA'
eumetsat_hotspot_gdf['satellite_sensor_product'] = eumetsat_hotspot_gdf['satellite']+'_'+eumetsat_hotspot_gdf['sensor']+'_EUMETSAT'
dea_hotspot_gdf['satellite_sensor_product'] = dea_hotspot_gdf['satellite']+'_'+dea_hotspot_gdf['sensor']+'_'+dea_hotspot_gdf['product']

In [ ]:
dea_hotspot_gdf['product'].unique()

In [ ]:
esa_hotspot_gdf['satellite_sensor_product'].unique()


In [ ]:
eumetsat_hotspot_gdf['satellite_sensor_product'].unique()

In [ ]:
dea_hotspot_gdf['satellite_sensor_product'].unique()

#### Plotting for all satellite sensor products from ESA, EUMETSAT and DEA 

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(20, 12))
plot(eumetsat_hotspot_gdf, 'F1_Fire_pixel_radiance', axs[0],  bbox, cmap='jet', title=f'Eumetsat FRP')
plot(esa_hotspot_gdf, 'F1_Fire_pixel_radiance', axs[1],  bbox, cmap='jet', title=f'ESA FRP')
plot(dea_hotspot_gdf, 'power', axs[2],  bbox, cmap='jet', title=f'DEA Hotspot')

### Subsetting Hotspot/FRP dataframes using column names and values
The sample for dea query arguments shows that we want specific satellite_sensor_product `TERRA_MODIS_SRSS` on `solar_date_range` 2019-11-1 and 2019-11-2 with `power` value greater than 10\ and `confidence` value greater than 40. We can use combination of many column names, value threshold to subset the dataframes to meet the analytic requirement of the users. \
The query are specified as `key: column names and values: (operator, column values threshold)` for all column names, except for `datetime fields: (start_date, end_date) `
```sample_query_arguments = {
    'satellite_sensor_product' : ('==', "'TERRA_MODIS_SRSS'"),  # remote satellite_sensor_product key if you want to include all the sensor products.
    'solar_date_range': (pd.to_datetime('2019-11-1'), pd.to_datetime('2019-11-2')),  # start and end date (will look inclusive of both the dates)
    'power': ('>', '10.'),
    'confidence': ('>', '40.'),
}```

In [ ]:
# generate all the individual satellite_sensor_product for dea hotspot for parameters specified in dea_hotspot_query_parameters
dea_hotspots_subsets_dict = {}
for prd in dea_hotspot_gdf['satellite_sensor_product'].unique():
    dea_hotspot_query_arguments = {
        **dea_hotspot_query_parameters,
        'satellite_sensor_product': ('==', f"'{prd}'")
    }
    dea_hotspot_subset = query_sv3t_gpd(dea_hotspot_gdf, dea_hotspot_query_arguments)
    filename = f"dea_hotspot_{prd.lower()}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.geojson"
    if not dea_hotspot_subset.empty:
        dea_hotspots_subsets_dict[prd] = dea_hotspot_subset
        # dea_hotspot_subset.to_file(filename, driver='GeoJSON')   # uncomment if you want to save to use with QGIS or other software

In [ ]:
# generate all the individual satellite_sensor_product for EUMETSAT FRP parameters specified in s3_frp_query_parameters
eumetsat_frp_subset_dict = {}
for prd in eumetsat_hotspot_gdf['satellite_sensor_product'].unique():
    eumetsat_hotspot_query_arguments = {
        **s3_frp_query_parameters,
        'satellite_sensor_product': ('==', f"'{prd}'")
    }
    filename = f"eumetsat_frp_{prd.lower()}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.geojson"
    eumetsat_frp_subset = query_sv3t_gpd(eumetsat_hotspot_gdf, eumetsat_hotspot_query_arguments)
    if not eumetsat_frp_subset.empty:
        eumetsat_frp_subset_dict[prd] = eumetsat_frp_subset
        # eumetsat_frp_subset.to_file(filename, driver='GeoJSON')  # uncomment if you want to save to use with QGIS or other software

In [ ]:
# list all the satellite_sensor_product for ESA FRP
# generate all the individual satellite_sensor_product for EUMETSAT FRP parameters specified in s3_frp_query_parameters
esa_frp_subset_dict = {}
for prd in esa_hotspot_gdf['satellite_sensor_product'].unique():
    esa_hotspot_query_arguments = {
        **s3_frp_query_parameters,
        'satellite_sensor_product': ('==', f"'{prd}'")
    }
    filename = f"esa_frp_{prd.lower()}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.geojson"
    esa_frp_subset = query_sv3t_gpd(esa_hotspot_gdf, esa_hotspot_query_arguments)
    if not esa_frp_subset.empty:
        esa_frp_subset_dict[prd] = esa_frp_subset
        # esa_frp_subset.to_file(filename, driver='GeoJSON') # uncomment if you want to save to use with QGIS or other software

In [ ]:
dea_hotspots_subsets_dict['NOAA 19_AVHRR_SRSS'].head()

In [ ]:
dea_hotspots_subsets_dict.keys()

In [ ]:
esa_frp_subset_dict.keys()

#### DEA Hotspot plots for individual satellite_sensor_product

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(20, 12))
esa_sensor_prd = 'SENTINEL_3B_SLSTR_ESA'
eumet_sensor_prd = 'SENTINEL_S3B_SLSTR_EUMETSAT'
dea_sensor_prd = 'NOAA 20_VIIRS_SRSS'
# bbox = (146., -35., 152., -30.)    # use smaller subset for zoom into a region of interest
plot(eumetsat_frp_subset_dict[eumet_sensor_prd], 'F1_Fire_pixel_radiance', axs[0],  bbox, cmap='jet', title=f'Eumetsat {eumet_sensor_prd} FRP')
plot(esa_frp_subset_dict[esa_sensor_prd], 'F1_Fire_pixel_radiance', axs[1],  bbox, cmap='jet', title=f'ESA {esa_sensor_prd} FRP')
plot(dea_hotspots_subsets_dict[dea_sensor_prd], 'longitude', axs[2],  bbox, cmap='jet', title=f'DEA {dea_sensor_prd} Hotspot')